In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
import tensorflow as tf 
import keras
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers


In [2]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [4]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [5]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 126, 64)           1216      
                                                                 
 conv1d_1 (Conv1D)           (None, 124, 64)           12352     
                                                                 
 dropout (Dropout)           (None, 124, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 62, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3968)              0         
                                                                 
 dense (Dense)               (None, 100)               396900    
                                                        

In [6]:
history = model.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=100,callbacks=callbacks)

Epoch 1/200
265/265 [==============================] - 16s 57ms/step - loss: 1.2157 - accuracy: 0.7290 - val_loss: 0.4772 - val_accuracy: 0.9032
Epoch 2/200
265/265 [==============================] - 16s 59ms/step - loss: 0.3132 - accuracy: 0.9200 - val_loss: 0.3227 - val_accuracy: 0.9334
Epoch 3/200
265/265 [==============================] - 16s 62ms/step - loss: 0.2041 - accuracy: 0.9465 - val_loss: 0.2642 - val_accuracy: 0.9406
Epoch 4/200
265/265 [==============================] - 17s 65ms/step - loss: 0.1407 - accuracy: 0.9611 - val_loss: 0.2267 - val_accuracy: 0.9462
Epoch 5/200
265/265 [==============================] - 18s 68ms/step - loss: 0.1069 - accuracy: 0.9695 - val_loss: 0.2114 - val_accuracy: 0.9505
Epoch 6/200
265/265 [==============================] - 18s 69ms/step - loss: 0.0857 - accuracy: 0.9747 - val_loss: 0.2189 - val_accuracy: 0.9420
Epoch 7/200
265/265 [==============================] - 15s 58ms/step - loss: 0.0655 - accuracy: 0.9796 - val_loss: 0.1981 - val_ac

In [7]:
loss, accuracy = model.evaluate(X_validation, y_validation, verbose=True)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy: {:.4f}".format(accuracy))

207/207 [==============================] - 1s 4ms/step - loss: 0.1914 - accuracy: 0.9557
Validation Accuracy:  0.9557
117/117 [==============================] - 1s 4ms/step - loss: 0.8415 - accuracy: 0.8930
Testing Accuracy: 0.8930
